In [2]:
from ultralytics import YOLO
import json
import pandas as pd
from yolo_config import Config
import os

## Agregate Folds

In [4]:
dataset = 'InBreast_Yolov8'

confusion_matrix_keys = [f'CM({i},{j})' for i in range(3) for j in range(3)]

keys = ['metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 
        'metrics/mAP50-95(B)', 'fitness'] + confusion_matrix_keys

for model_size in Config.MODEL_SIZES:
    for conf_threshold in Config.THRESHOLDS:
        aggregated_results = {k:0 for k in keys}

        model_dir = f'{model_size}_conf={conf_threshold}'
        model_path = os.path.join(Config.RUNS_DIR, Config.DATASET, model_dir)

        for fold in range(1,11):
            fold_dir = f'fold_{fold}'
            results_path = os.path.join(model_path, fold_dir, 'val', 'results.json')
            results = json.load(open(results_path))
            for k in keys:
                aggregated_results[k] += results[k]
            
        aggregated_results = {k:v/10 for k,v in aggregated_results.items()}
        aggregated_results_path = os.path.join(model_path, 'results.json')

        with open(aggregated_results_path, 'w+') as file:
            json.dump(aggregated_results, file)

        print(aggregated_results)
        
        break # Threshold
    break # Model Size

{'metrics/precision(B)': 0.0, 'metrics/recall(B)': 0.0, 'metrics/mAP50(B)': 0.0, 'metrics/mAP50-95(B)': 0.0, 'fitness': 0.0, 'CM(0,0)': 0.0, 'CM(0,1)': 0.0, 'CM(0,2)': 0.0, 'CM(1,0)': 0.0, 'CM(1,1)': 0.0, 'CM(1,2)': 0.0, 'CM(2,0)': 0.8, 'CM(2,1)': 0.6, 'CM(2,2)': 0.0}


## Summarize Results

In [5]:
all_results = []
model_names = []

for model_size in Config.MODEL_SIZES:
    for conf_threshold in Config.THRESHOLDS:
        model_name = f'{model_size}_conf={conf_threshold}'
        aggregated_results_path = os.path.join(Config.RUNS_DIR, Config.DATASET, 
                                               model_name, 'results.json')

        all_results.append(json.load(open(aggregated_results_path)))
        model_names.append(model_name)

        break # Threshold
    break # Model Size

all_results_path = os.path.join(Config.RUNS_DIR, Config.DATASET, 'all_results.csv')
all_results = pd.DataFrame(all_results, index=model_names)


In [6]:
all_results.to_csv(all_results_path)
all_results

,metrics/precision(B),metrics/recall(B),metrics/mAP50(B),metrics/mAP50-95(B),fitness,"CM(0,0)","CM(0,1)","CM(0,2)","CM(1,0)","CM(1,1)","CM(1,2)","CM(2,0)","CM(2,1)","CM(2,2)"
yolov8n.pt_conf=0.1,0,0,0,0,0,0,0,0,0,0,0,0.8,0.6,0
